# OpenSearch Access Setting

## Prerequisites
- For this hands-on exercise, the role (Role) executing the notebook must have the following permissions added.
    - AmazonOpenSearchServiceFullAccess
    - AmazonSSMFullAccess
- Amazon OpenSearch's domain had to be created and id/pw was stored in AWS Secret Manager

### Load library

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os
module_path = "../../../.."
sys.path.append(os.path.abspath(module_path))

In [ ]:
import boto3
import uuid
import botocore
import time

In [ ]:
from utils.ssm import parameter_store

### Extract id and pw from AWS Secret Manager

In [ ]:
secrets_manager = boto3.client("secretsmanager")
response = secrets_manager.get_secret_value(
    SecretId = 'opensearch_user_password'
)
secrets_string = response.get('SecretString')
secrets_dict = eval(secrets_string)

opensearch_user_id = secrets_dict['es.net.http.auth.user']
opensearch_user_password = secrets_dict['pwkey']


### Obtain opensearch_domain_endpoint from OpenSearch Console

In [ ]:
opensearch_domain_endpoint =  "Paste domain url from OpenSearch Console"
opensearch_domain_endpoint = "https://search-rag-hol-mydomain-ehghqwhortwcjrvsk2tmj3edce.us-west-2.es.amazonaws.com"


### Store id, pw, opensearch_domain_endpoint into parameter store

In [ ]:
# 2. Store OpenSearch authentication information in SSM

region=boto3.Session().region_name
pm = parameter_store(region)

pm.put_params(
    key="opensearch_domain_endpoint",
    value=f'{opensearch_domain_endpoint}',
    overwrite=True,
    enc=False
)

pm.put_params(
    key="opensearch_user_id",
    value=f'{opensearch_user_id}',
    overwrite=True,
    enc=False
)

pm.put_params(
    key="opensearch_user_password",
    value=f'{opensearch_user_password}',
    overwrite=True,
    enc=True
)


In [ ]:
print (pm.get_params(key="opensearch_domain_endpoint", enc=False))
print (pm.get_params(key="opensearch_user_id", enc=False))
print (pm.get_params(key="opensearch_user_password", enc=True))